In [ ]:
!pip install pathway bokeh --quiet

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
df=pd.read_csv('dataset.csv')

In [ ]:
#making a list of parking lots to get plots one by one
lotss=df['SystemCodeNumber'].unique()
lots=lotss[0:14]
lots

In [ ]:
plots=[]
for lot in lots:
  df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],  #combining date and time columns and converting it in datetime format
                                  format='%d-%m-%Y %H:%M:%S')
  df_lot=df[df['SystemCodeNumber']==lot]   #making dataframe for one lot at a time
  df_lot = df_lot.sort_values('Timestamp').reset_index(drop=True)  #sorting time

  df_lot[["Timestamp", "Occupancy", "Capacity","SystemCodeNumber"]].to_csv("parking_stream.csv", index=False)  #filtering out the columns that are being used for model 1
  class ParkingSchema(pw.Schema):
    Capacity:int
    Occupancy:int
    Timestamp:str
    SystemCodeNumber:str

  filename = f"parking_stream_{lot}.csv"
  df_lot[["Timestamp", "Occupancy", "Capacity", "SystemCodeNumber"]].to_csv(filename, index=False)
  data = pw.demo.replay_csv(filename, schema=ParkingSchema, input_rate=1000)  # Load the data as a simulated stream using Pathway's replay_csv function
  fmt = "%Y-%m-%d %H:%M:%S"
  data_with_time = data.with_columns(
      t = data.Timestamp.dt.strptime(fmt),   #containes full datetime
      day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00"),  #contains only day date
      hour = data.Timestamp.dt.strptime(fmt).dt.hour(),  #contains hour
      day_of_week = data.Timestamp.dt.strptime(fmt).dt.weekday(),   #assigns monday: 0,...,sunday: 6
      occupancy_rate = data.Occupancy / data.Capacity
  )
  def time_of_day_weight(hour):    #the reason for choosing these categories is explained in the report

    if 11 <= hour < 14:
        return 1.0    # Midday
    elif 14 <= hour < 17:
        return 0.7    # Evening
    else:
        return 0.4    # Morning

  def weekday_weight(day_of_week):   #the reason for choosing these categories is explained in the report

    if day_of_week < 5:
        return 1.0    # Weekday
    else:
        return 0.7    # Weekend

  def pricing_fn(occ_rate, tod_weight, wd_weight):
    return 10.0 + 2.0 * occ_rate * tod_weight * wd_weight

  import datetime

  data_with_price=(
     data_with_time.with_columns(
         tod_weight = pw.apply(time_of_day_weight, data_with_time.hour),
         wd_weight = pw.apply(weekday_weight, data_with_time.day_of_week)



        )
    .with_columns(
        price=pw.apply(
        pricing_fn,
        data_with_time.occupancy_rate,
        pw.this.tod_weight,
        pw.this.wd_weight
    )

    )


)

  plot_table = data_with_price.select(data_with_price.t, data_with_price.price,data_with_price.SystemCodeNumber)
  pn.extension()

#writing bokeh plot fn
  def price_plotter(source):

    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title=f"Pathway: Daily Parking Price of  {lot}",
        x_axis_type="datetime",
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.scatter("t", "price", source=source, size=6, color="red")

    return fig


# - 'sorting_col="t"' ensures the data is plotted in time order
  viz = plot_table.plot(price_plotter, sorting_col="t")
  plots.append(pn.Column(f"Lot: {lot}", viz.servable()))


dashboard = pn.Column(*plots)   ## Create a dashboard container that holds all individual plots stored in 'plots'
dashboard.servable()   # Make the dashboard servable


In [ ]:
pw.run()